# Analisi - Grafi

In [1]:
import pickle

import geopandas as gpd
import networkx as nx
import osmnx as ox

from my_paths import *
import Graph_functions

## Sport e tempo libero

Utilizzeremo i seguenti dati:
1. Parchi
2. impianti sportivi
3. aree gioco
Saranno i nostri "poi", cioè "Point of Interest" che verranno aggiunti al Grafo della rete Ciclabile/Stradale per la ricerca di percorsi

Carichiamo il Grafo di base della rete stradale/ciclabile

In [2]:
# Carichiamo Graph Ciclabili
with open(PATH_RETE_CICLABILE_COMPLETA_PICKLE_CLEAN, "rb") as f:
    G_strade_ciclabili = pickle.load(f)

### Aggiunta di Parchi come **"poi"**

1. Carichiamo il file dei parchi da "clean"
2. Utilizziamo **"connect_poi_nodes_to_graph()"** per connettere il geodataframe come "poi" al grafo delle strade/ciclabili.

In [3]:
gdf_parchi = gpd.read_file(PATH_PARCHI_CLEAN)
G_compose_merged_parchi = Graph_functions.connect_poi_nodes_to_graph(G_strade_ciclabili, gdf_parchi,
                                                                     tipo = "parchi",
                                                                     poi = True,
                                                                     artificial=True)

## Utilizziamo algoritmo di **Steiner Tree**

Una volta aggiunti i "poi" al grafo delle strade/ciclabili, eseguiamo l'algoritmo che troverà i percorsi minimi per connettere tutti i "poi" tra di loro con il minor "peso" possibile.  
I pesi sono stati già accuratamente inseriti sugli archi delle strade, mentre quelli delle ciclabili e degli archi all'interno dei poligoni dei "poi" sono = 0, in modo da preferire sempre quei percorsi quando possibile (sfruttare ciclabili pre-esistenti il più possibile).

In [4]:
poi_nodes = [n for n, d in G_compose_merged_parchi.nodes(data=True) if d.get("tipo") == "parchi"]

components = list(nx.connected_components(G_compose_merged_parchi))
largest_cc = max(components, key=len)

G_sub = G_compose_merged_parchi.subgraph(largest_cc).copy()
poi_sub = [n for n in poi_nodes if n in G_sub.nodes]

for u, v, data in G_sub.edges(data=True):
    if 'weight' not in data or not isinstance(data['weight'], (int, float)):
        data['weight'] = 1  # o un peso di default

from networkx.algorithms.approximation import steiner_tree

G_sport_tempo_libero = steiner_tree(G_sub, terminal_nodes=poi_sub, weight='weight')

Risultato lo salviamo in Clean di Analisi (per ora anche il pickle in staging) in geojson pronto da visualizzare

In [ ]:
with open(PATH_SPORT_TEMPO_LIBERO_ANALISI_PICKLE_STAGING, "wb") as f:
    pickle.dump(G_sport_tempo_libero, f)

gdf_steiner = ox.graph_to_gdfs(G_sport_tempo_libero, edges=True, nodes=False)
gdf_steiner.to_file(PATH_SPORT_TEMPO_LIBERO_ANALISI_CLEAN, driver="GeoJSON")